# Creating German Datasets

## Mount Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## GPU Info

In [2]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Fri Jan 31 12:16:49 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   28C    P0             43W /  400W |       0MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

## Imports

In [3]:
!pip install -U -q bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 83.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 67.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 35.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 85.2 MB/s eta 0:00:00


In [6]:
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments, AutoModelForMaskedLM, pipeline
import torch
import pandas as pd
import numpy as np
from pprint import pprint
from math import e
from tqdm import tqdm

In [5]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print('Using {} device'.format(device))

Using cuda device


In [9]:
! huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
The token `sonic` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `sonic`


### Loading Data

In [35]:
data_path = '../data/'

# file with the original study data
data_file = 'tibblin_data_en_de.csv'

In [36]:
data_en = pd.read_csv(data_path + data_file)
data_en_ext = pd.read_csv(data_path+'tibblin_data_en_extended.csv')
data_en_ext = data_en_ext.set_index('item_no')
data_en.head()

,item_no,role_noun,role_noun_EN,role_noun_DE,Phrase1,Phrase2,Phrase1_EN,Phrase2_EN,Phrase1_DE,Phrase2_DE,Continuation,continuation_EN,exp_answer,déterminant,filler_type
0,1,cardinaux,cardinals,Kardinäle,Les cardinaux montaient dans le bus.,"Etant donné le soleil, plusieurs hommes portai...",The cardinals were getting on the bus.,"Given the sun, several of the men were wearing...",Die Kardinäle stiegen in den Bus.,Angesichts der Sonne trugen mehrere der Männer...,h,men,yes,plusieurs,lexical_gender
1,2,Femme de chambre,chambermaids,Zimmermädchen,Les femmes de chambres traversaient le hall.,"Etant donné le mauvais temps, la majorité des ...",The chambermaids were crossing the hall.,"Given the bad weather, most of the women were ...",Die Zimmermädchen durchquerten die Halle.,Aufgrund des schlechten Wetters trugen die mei...,f,women,yes,majorité,lexical_gender
2,3,Frères,brothers,Brüder,Les frères montaient dans le car.,"Etant donné le beau temps, quelques hommes por...",The brothers were getting on the bus.,"Given the good weather, a few of the men were ...",Die Brüder stiegen in den Reisebus.,Bei schönem Wetter trugen einige der Männer Sh...,h,men,yes,plusieurs,lexical_gender
3,4,Garçons d'étage,bellboys,Etagenjungen,Les garçons d'étage marchaient dans la ville.,"Après une très longue journée, la plupart des ...",The bellboys were walking in the city.,"After a very long day, most of the men seemed ...",Die Etagenjungen gingen durch die Stadt.,Nach einem sehr langen Tag schienen die meiste...,h,men,yes,plupart,lexical_gender
4,5,Grand-mères,grandmothers,Großmütter,Les grand-mères attendaient sur un banc.,"A cause du temps nuageux, un des hommes portai...",The grandmothers were waiting on a bench.,"Because of the cloudy weather, one of the men ...",Die Großmütter warteten auf einer Bank.,Wegen des wolkigen Wetters trug einer der Männ...,h,men,non,un,lexical_gender


In [37]:
# extract German data
selected_columns = ['item_no', 'role_noun_DE', 'Phrase1_DE', 'Phrase2_DE']
data_de = data_en[selected_columns]
data_de.set_index('item_no', inplace=True)
data_de.head()

,role_noun_DE,Phrase1_DE,Phrase2_DE
item_no,,,
1,Kardinäle,Die Kardinäle stiegen in den Bus.,Angesichts der Sonne trugen mehrere der Männer...
2,Zimmermädchen,Die Zimmermädchen durchquerten die Halle.,Aufgrund des schlechten Wetters trugen die mei...
3,Brüder,Die Brüder stiegen in den Reisebus.,Bei schönem Wetter trugen einige der Männer Sh...
4,Etagenjungen,Die Etagenjungen gingen durch die Stadt.,Nach einem sehr langen Tag schienen die meiste...
5,Großmütter,Die Großmütter warteten auf einer Bank.,Wegen des wolkigen Wetters trug einer der Männ...


In [38]:
data_de = data_de.rename(columns={'role_noun_DE': 'ante_noun',
                                  'role_noun_gender': 'ante_gender',
                                  'Phrase1_DE': 'phrase1',
                                  'Phrase2_DE': 'phrase2'})

### Extending & Editing Data

#### Phrase 2

In [39]:
def mask_cut_gender(phrase):
    """ MASKED: return phrase with gendered term masked
        CUT: return phrase up until the point where gendered term is mentioned"""

    mf = ['Männer', 'Frauen']
    tokens = phrase.split()

    if mf[0] in tokens:
        end_pos = phrase.find(mf[0])
        cut = phrase[:end_pos].strip()
        masked = phrase.replace(mf[0], '[MASK]')
        gender = 'm'
    elif mf[1] in tokens:
        end_pos = phrase.find(mf[1])
        cut = phrase[:end_pos].strip()
        masked = phrase.replace(mf[1], '[MASK]')
        gender = 'f'
    else:
        end_pos = phrase.find('Jungen')
        cut = phrase[:end_pos].strip()
        masked = phrase.replace('Jungen', '[MASK]')
        gender = 'm'

    return masked, cut, gender

In [40]:
mask_cut_gender("Nach einiger Zeit schienen einige Männer umkehren zu wollen.")

('Nach einiger Zeit schienen einige [MASK] umkehren zu wollen.',
 'Nach einiger Zeit schienen einige',
 'm')

In [41]:
data_de['phrase2_masked'] = data_de['phrase2'].apply(lambda x: mask_cut_gender(x)[0])
data_de['phrase2_cut'] = data_de['phrase2'].apply(lambda x: mask_cut_gender(x)[1])
data_de['coref_gender'] = data_de['phrase2'].apply(lambda x: mask_cut_gender(x)[2])

In [42]:
# add antecedent gender from English data
data_de['ante_gender'] = data_en_ext['role_noun_gender']

In [43]:
data_de.head()

,ante_noun,phrase1,phrase2,phrase2_masked,phrase2_cut,coref_gender,ante_gender
item_no,,,,,,,
1,Kardinäle,Die Kardinäle stiegen in den Bus.,Angesichts der Sonne trugen mehrere der Männer...,Angesichts der Sonne trugen mehrere der [MASK]...,Angesichts der Sonne trugen mehrere der,m,m
2,Zimmermädchen,Die Zimmermädchen durchquerten die Halle.,Aufgrund des schlechten Wetters trugen die mei...,Aufgrund des schlechten Wetters trugen die mei...,Aufgrund des schlechten Wetters trugen die mei...,f,f
3,Brüder,Die Brüder stiegen in den Reisebus.,Bei schönem Wetter trugen einige der Männer Sh...,Bei schönem Wetter trugen einige der [MASK] Sh...,Bei schönem Wetter trugen einige der,m,m
4,Etagenjungen,Die Etagenjungen gingen durch die Stadt.,Nach einem sehr langen Tag schienen die meiste...,Nach einem sehr langen Tag schienen die meiste...,Nach einem sehr langen Tag schienen die meiste...,m,m
5,Großmütter,Die Großmütter warteten auf einer Bank.,Wegen des wolkigen Wetters trug einer der Männ...,Wegen des wolkigen Wetters trug einer der [MAS...,Wegen des wolkigen Wetters trug einer der,m,f


#### Phrase 1

In [44]:
p1s_masked = []

for i, row in data_de.iterrows():
    phrase1_masked = row['phrase1'].replace(row['ante_noun'], '[MASK]')
    phrase1_masked = phrase1_masked.replace('[MASK]n', '[MASK]') # error due to 'bridesmaid'
    p1s_masked.append(phrase1_masked)

p1s_masked[:5]

['Die [MASK] stiegen in den Bus.',
 'Die [MASK] durchquerten die Halle.',
 'Die [MASK] stiegen in den Reisebus.',
 'Die [MASK] gingen durch die Stadt.',
 'Die [MASK] warteten auf einer Bank.']

In [45]:
data_de['phrase1_masked'] = p1s_masked

In [46]:
data_de.head()

,ante_noun,phrase1,phrase2,phrase2_masked,phrase2_cut,coref_gender,ante_gender,phrase1_masked
item_no,,,,,,,,
1,Kardinäle,Die Kardinäle stiegen in den Bus.,Angesichts der Sonne trugen mehrere der Männer...,Angesichts der Sonne trugen mehrere der [MASK]...,Angesichts der Sonne trugen mehrere der,m,m,Die [MASK] stiegen in den Bus.
2,Zimmermädchen,Die Zimmermädchen durchquerten die Halle.,Aufgrund des schlechten Wetters trugen die mei...,Aufgrund des schlechten Wetters trugen die mei...,Aufgrund des schlechten Wetters trugen die mei...,f,f,Die [MASK] durchquerten die Halle.
3,Brüder,Die Brüder stiegen in den Reisebus.,Bei schönem Wetter trugen einige der Männer Sh...,Bei schönem Wetter trugen einige der [MASK] Sh...,Bei schönem Wetter trugen einige der,m,m,Die [MASK] stiegen in den Reisebus.
4,Etagenjungen,Die Etagenjungen gingen durch die Stadt.,Nach einem sehr langen Tag schienen die meiste...,Nach einem sehr langen Tag schienen die meiste...,Nach einem sehr langen Tag schienen die meiste...,m,m,Die [MASK] gingen durch die Stadt.
5,Großmütter,Die Großmütter warteten auf einer Bank.,Wegen des wolkigen Wetters trug einer der Männ...,Wegen des wolkigen Wetters trug einer der [MAS...,Wegen des wolkigen Wetters trug einer der,m,f,Die [MASK] warteten auf einer Bank.


#### Save masked & cut data
The data are saved here to allow for some initial editing of errors occuring during the masking process.

In [ ]:
data_de.to_csv(data_path+'tibblin_data_de_raw.csv')

In [48]:
data_de = pd.read_csv(data_path+"tibblin_data_de_revised.csv")

### Creating dataset with coreferents

In [49]:
pers_nouns_pl = ["Männer", "Frauen", "Personen"]
pers_genders = ['m', 'f', 'n']

In [50]:
long_data = []

# create a new dataframe with log_probs
for i, row in data_de.iterrows():
    for j in range(len(pers_genders)): # range 3 bc we're testing for 'men', 'women', 'people'
        new_row = row.copy()
        new_row["coref"] = pers_nouns_pl[j]
        new_row["coref_gender"] = pers_genders[j]
        new_row["phrases_full"] = row['phrase1'] + ' ' + row['phrase2_masked'].replace('[MASK]', pers_nouns_pl[j])
        new_row["phrases_cut"] = row['phrase1'] + ' ' + row['phrase2_cut'] + ' ' + pers_nouns_pl[j]
        long_data.append(new_row)

if len(long_data) == len(data_de)*3:
    print(True)
else:
    print(len(long_data))
    print(len(data_de))

True


In [51]:
data_de_long = pd.DataFrame(long_data)

In [52]:
print(data_de_long.iloc[[2]].loc[:,'phrase2_cut'].item())

Angesichts der Sonne trugen mehrere der


In [28]:
for i in data_de_long['phrases_full'].head():
    print(i)

Die Kardinäle stiegen in den Bus. Angesichts der Sonne trugen mehrere der Männer einen Hut.
Die Kardinäle stiegen in den Bus. Angesichts der Sonne trugen mehrere der Frauen einen Hut.
Die Kardinäle stiegen in den Bus. Angesichts der Sonne trugen mehrere der Personen einen Hut.
Die Zimmermädchen durchquerten die Halle. Aufgrund des schlechten Wetters trugen die meisten der Männer eine Mütze.
Die Zimmermädchen durchquerten die Halle. Aufgrund des schlechten Wetters trugen die meisten der Frauen eine Mütze.


#### Save long data

In [ ]:
data_de_long.to_csv(data_path+'tibblin_de_long.csv', index=False)

### Adding antecedents to dataset

#### Import triplets

In [53]:
triplets_de_pl = pd.read_csv(data_path+'triplets_plural_DE.csv')

In [54]:
triplets_de_pl['number'] = 'PL'

In [55]:
triplets_de_pl.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   masc        10 non-null     object
 1   fem         10 non-null     object
 2   coord_f     10 non-null     object
 3   coord_m     10 non-null     object
 4   star        10 non-null     object
 5   colon       10 non-null     object
 6   underscore  10 non-null     object
 7   big_i       10 non-null     object
 8   number      10 non-null     object
dtypes: object(9)
memory usage: 852.0+ bytes



#### Create Templates


Mask the first sentences

In [56]:
templates_de = data_de[['item_no','phrase1_masked','phrase2_masked', 'phrase2_cut']]
templates_de.loc[:,'number'] = 'PL'

/tmp/ipykernel_126183/712960925.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  templates_de.loc[:,'number'] = 'PL'


In [57]:
templates_de

,item_no,phrase1_masked,phrase2_masked,phrase2_cut,number
0,1,Die [MASK] stiegen in den Bus.,Angesichts der Sonne trugen mehrere der [MASK]...,Angesichts der Sonne trugen mehrere der,PL
1,2,Die [MASK] durchquerten die Halle.,Aufgrund des schlechten Wetters trugen die mei...,Aufgrund des schlechten Wetters trugen die mei...,PL
2,3,Die [MASK] stiegen in den Reisebus.,Bei schönem Wetter trugen einige der [MASK] Sh...,Bei schönem Wetter trugen einige der,PL
3,4,Die [MASK] gingen durch die Stadt.,Nach einem sehr langen Tag schienen die meiste...,Nach einem sehr langen Tag schienen die meiste...,PL
4,5,Die [MASK] warteten auf einer Bank.,Wegen des wolkigen Wetters trug einer der [MAS...,Wegen des wolkigen Wetters trug einer der,PL
5,6,Die [MASK] gingen über den Platz.,Aufgrund der Hitze hatten mehrere der [MASK] k...,Aufgrund der Hitze hatten mehrere der,PL
6,7,Die Geschäftsleute waren schon bei der Arbeit.,"Man konnte sehen, dass mehrere der [MASK] wirk...","Man konnte sehen, dass mehrere der",PL
7,8,Die [MASK] gingen zum Supermarkt.,"Da schlechtes Wetter vorhergesagt wurde, hatte...","Da schlechtes Wetter vorhergesagt wurde, hatte...",PL
8,9,Die [MASK] warteten in der Nähe der Schranke.,Nach einiger Zeit schienen die meisten der [MA...,Nach einiger Zeit schienen die meisten der,PL
9,10,Die [MASK] gingen eilig hinaus.,"Man konnte sehen, dass die meisten der [MASK] ...","Man konnte sehen, dass die meisten der",PL


#### Save Templates

In [ ]:
templates_de.to_csv(data_path+'templates_pl_DE.csv', index=False)

In [27]:
templates_de = pd.read_csv(data_path+'templates_pl_DE_revised.csv')

#### Populate with antecedent triplets
These are the data used for coreferent generation. 

In [58]:
def populate_templates2(templates, triplets):
    pop_temp_rows = []

    for _, temp_row in templates.iterrows(): # all templates
        for _, trip_row in triplets.iterrows(): # all the triplets
            # replace the noun in the first sentence
            for gender in triplets.columns[:-1]:
                p1 = temp_row['phrase1_masked'].replace('[MASK]', trip_row[gender])
                ante_gender = gender
                phrases_cut = p1 + ' ' + temp_row['phrase2_cut']

                new_row = {'phrase1': p1,
                        'phrase2_cut': temp_row['phrase2_cut'],
                        'ante_noun': trip_row[gender],
                        'ante_gender': ante_gender,
                        'phrases_cut': phrases_cut}

                pop_temp_rows.append(new_row)

    return pd.DataFrame(pop_temp_rows)

In [59]:
data_for_next = populate_templates2(templates_de,
                                    triplets_de_pl)

In [60]:
data_for_next_small = populate_templates2(templates_de.iloc[0:2,:], triplets_de_pl)

In [30]:
data_for_next.head()

,phrase1,phrase2_cut,ante_noun,ante_gender,phrases_cut
0,Die Eigentümer stiegen in den Bus.,Angesichts der Sonne trugen mehrere der,Eigentümer,masc,Die Eigentümer stiegen in den Bus. Angesichts ...
1,Die Eigentümerinnen stiegen in den Bus.,Angesichts der Sonne trugen mehrere der,Eigentümerinnen,fem,Die Eigentümerinnen stiegen in den Bus. Angesi...
2,Die Eigentümerinnen und Eigentümer stiegen in ...,Angesichts der Sonne trugen mehrere der,Eigentümerinnen und Eigentümer,coord_f,Die Eigentümerinnen und Eigentümer stiegen in ...
3,Die Eigentümer und Eigentümerinnen stiegen in ...,Angesichts der Sonne trugen mehrere der,Eigentümer und Eigentümerinnen,coord_m,Die Eigentümer und Eigentümerinnen stiegen in ...
4,Die Eigentümer*innen stiegen in den Bus.,Angesichts der Sonne trugen mehrere der,Eigentümer*innen,star,Die Eigentümer*innen stiegen in den Bus. Anges...


In [35]:
data_for_next.to_csv(data_path+'data_for_next_DE_PL.csv', index=False)
data_for_next_small.to_csv(data_path+'data_for_next_DE_PL_small.csv', index=False)

In [37]:
data_for_next_small['phrases_cut'].iloc[0]

'Die Eigentümer stiegen in den Bus. Angesichts der Sonne trugen mehrere der'

#### Populate with antecedents and coreferents
This dataset will be used for coreferent probability experiments.

In [61]:
triplets_de_pl.columns[:-1]

Index(['masc', 'fem', 'coord_f', 'coord_m', 'star', 'colon', 'underscore',
       'big_i'],
      dtype='object')

In [63]:
def populate_templates(templates, triplets, pers_nouns, pers_genders):
    pop_temp_rows = []

    for _, temp_row in templates.iterrows(): # all templates
        for _, trip_row in triplets.iterrows(): # all the triplets
            # replace the noun in the first sentence
            for gender in triplets.columns[:-1]:
                for i in range(len(pers_genders)):
                    p1 = temp_row['phrase1_masked'].replace('[MASK]', trip_row[gender])
                    p2 = temp_row['phrase2_masked'].replace('[MASK]', pers_nouns[i])
                    ante_gender = gender
                    #ante_gender = gender[0] # gender is a full word
                    coref_gender = pers_genders[i]
                    phrases_full = p1 + ' '+ p2
                    phrases_cut = p1 + ' ' + temp_row['phrase2_cut'] +' ' + pers_nouns[i]

                    new_row = {'phrase1': p1,
                            'phrase2': p2,
                            'phrase2_cut': temp_row['phrase2_cut'],
                            'ante_noun': trip_row[gender],
                            'ante_gender': ante_gender,
                            'coref_noun': pers_nouns[i],
                            'coref_gender': coref_gender,
                            'phrases_full': phrases_full,
                            'phrases_cut': phrases_cut}

                    pop_temp_rows.append(new_row)

    return pd.DataFrame(pop_temp_rows)

In [64]:
data_from_templates_pl = populate_templates(templates_de,
                                            triplets_de_pl,
                                            pers_nouns_pl,
                                            pers_genders)

In [65]:
print(data_from_templates_pl['phrases_cut'][190:210])

190    Die SchiedsrichterInnen stiegen in den Bus. An...
191    Die SchiedsrichterInnen stiegen in den Bus. An...
192    Die Tierärzte stiegen in den Bus. Angesichts d...
193    Die Tierärzte stiegen in den Bus. Angesichts d...
194    Die Tierärzte stiegen in den Bus. Angesichts d...
195    Die Tierärztinnen stiegen in den Bus. Angesich...
196    Die Tierärztinnen stiegen in den Bus. Angesich...
197    Die Tierärztinnen stiegen in den Bus. Angesich...
198    Die Tierärztinnen und Tierärzte stiegen in den...
199    Die Tierärztinnen und Tierärzte stiegen in den...
200    Die Tierärztinnen und Tierärzte stiegen in den...
201    Die Tierärzte und Tierärztinnen stiegen in den...
202    Die Tierärzte und Tierärztinnen stiegen in den...
203    Die Tierärzte und Tierärztinnen stiegen in den...
204    Die Tierärzt*innen stiegen in den Bus. Angesic...
205    Die Tierärzt*innen stiegen in den Bus. Angesic...
206    Die Tierärzt*innen stiegen in den Bus. Angesic...
207    Die Tierärzt:innen stieg

In [66]:
data_from_templates_pl.ante_gender.value_counts()

ante_gender
masc          1320
fem           1320
coord_f       1320
coord_m       1320
star          1320
colon         1320
underscore    1320
big_i         1320
Name: count, dtype: int64

In [67]:
len(data_from_templates_pl)

10560

In [44]:
data_from_templates_pl.tail()

,phrase1,phrase2,phrase2_cut,ante_noun,ante_gender,coref_noun,coref_gender,phrases_full,phrases_cut
10555,Die Archäolog_innen gingen durch den Schneesturm.,Angesichts der Hitze trugen einige der Frauen ...,Angesichts der Hitze trugen einige der,Archäolog_innen,underscore,Frauen,f,Die Archäolog_innen gingen durch den Schneestu...,Die Archäolog_innen gingen durch den Schneestu...
10556,Die Archäolog_innen gingen durch den Schneesturm.,Angesichts der Hitze trugen einige der Persone...,Angesichts der Hitze trugen einige der,Archäolog_innen,underscore,Personen,n,Die Archäolog_innen gingen durch den Schneestu...,Die Archäolog_innen gingen durch den Schneestu...
10557,Die ArchäologInnen gingen durch den Schneesturm.,Angesichts der Hitze trugen einige der Männer ...,Angesichts der Hitze trugen einige der,ArchäologInnen,big_i,Männer,m,Die ArchäologInnen gingen durch den Schneestur...,Die ArchäologInnen gingen durch den Schneestur...
10558,Die ArchäologInnen gingen durch den Schneesturm.,Angesichts der Hitze trugen einige der Frauen ...,Angesichts der Hitze trugen einige der,ArchäologInnen,big_i,Frauen,f,Die ArchäologInnen gingen durch den Schneestur...,Die ArchäologInnen gingen durch den Schneestur...
10559,Die ArchäologInnen gingen durch den Schneesturm.,Angesichts der Hitze trugen einige der Persone...,Angesichts der Hitze trugen einige der,ArchäologInnen,big_i,Personen,n,Die ArchäologInnen gingen durch den Schneestur...,Die ArchäologInnen gingen durch den Schneestur...


#### Save Dataset 

In [46]:
data_from_templates_pl.to_csv(data_path+f'tibblin_populated_templates_pl_DE.csv', index=True)

In [45]:
# Example
data_from_templates_pl.iloc[23,]

,23
phrase1,Die EigentümerInnen stiegen in den Bus.
phrase2,Angesichts der Sonne trugen mehrere der Person...
phrase2_cut,Angesichts der Sonne trugen mehrere der
ante_noun,EigentümerInnen
ante_gender,big_i
coref_noun,Personen
coref_gender,n
phrases_full,Die EigentümerInnen stiegen in den Bus. Angesi...
phrases_cut,Die EigentümerInnen stiegen in den Bus. Angesi...
